#Tryout #4
Dropped learning data size to 10000 each class

In [1]:
import sys
import numpy as np
import torch
import torch.nn as nn
#from torch.autograd import Function
from torchvision import transforms, datasets
# import math

import torch.nn.functional as F
# import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, ConcatDataset
# import copy
import tqdm
from PIL import Image

# Load the data (extract)
import os
import zipfile
from pathlib import Path
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
if not (os.path.exists('tmp/train')):
  local_zip = '/content/drive/My Drive/datasets/dogscats/train.zip'
  zip_ref = zipfile.ZipFile(local_zip, 'r')
  zip_ref.extractall('/tmp')
  zip_ref.close()
  local_zip = '/content/drive/My Drive/datasets/dogscats/test.zip'
  zip_ref = zipfile.ZipFile(local_zip, 'r')
  zip_ref.extractall('/tmp')
  zip_ref.close()
import gc
gc.collect()

base_dir = '/tmp'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'test1')

train_fnames = os.listdir(train_dir)
validation_fnames = os.listdir(validation_dir)

train_cat_fnames = [os.path.join(train_dir, x) for x in train_fnames if x.startswith('cat')][:-2500]
train_dog_fnames = [os.path.join(train_dir, x) for x in train_fnames if x.startswith('dog')][:-2500]
validation_cat_fnames = [os.path.join(train_dir, x) for x in train_fnames if x.startswith('cat')][-500:]
validation_dog_fnames = [os.path.join(train_dir, x) for x in train_fnames if x.startswith('dog')][-500:]

del train_fnames
del validation_fnames
print('total training cat images:', len(train_cat_fnames))
print('total training dog images:', len(train_dog_fnames))
print('total validation cat images:', len(validation_cat_fnames))
print('total validation dog images:', len(validation_dog_fnames))

total training cat images: 10000
total training dog images: 10000
total validation cat images: 500
total validation dog images: 500


In [0]:

class CatDogDataset(Dataset):
    def __init__(self, file_list, label=-1, mode='train', transform = None):
        self.file_list = file_list
        self.label = label
        self.mode = mode
        self.transform = transform
        
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, idx):
        img = Image.open(self.file_list[idx])
        img
        if self.transform:
            img = self.transform(img)
            
        if self.mode == 'train':
            img = img.numpy()
            return img.astype('float32'), self.label
        else:
            img = img.numpy()
            return img.astype('float32'), self.label #file_list[idx]
        
data_transform = transforms.Compose([
    transforms.Resize((45,45)),
    transforms.Resize((450,450)),
    transforms.Grayscale(),
    transforms.ToTensor()
])

train_cats_ds = CatDogDataset(train_cat_fnames, 0, transform = data_transform)
train_dogs_ds = CatDogDataset(train_dog_fnames, 1, transform = data_transform)

train_catdogs_ds = ConcatDataset([train_cats_ds, train_dogs_ds])
dataloader = DataLoader(train_catdogs_ds, batch_size = 32, shuffle=True, num_workers=0)

validation_cats = CatDogDataset(validation_cat_fnames, 0, transform = data_transform)
validation_dogs = CatDogDataset(validation_dog_fnames, 1, transform = data_transform)

validation_catdogs = ConcatDataset([validation_cats, validation_dogs])
validation_size = len(validation_cat_fnames) + len(validation_dog_fnames)
validation_dataloader = DataLoader(validation_catdogs, batch_size = int(validation_size / 100), shuffle=False, num_workers=0)

In [4]:
samples, labels = iter(dataloader).next()
print(samples.shape)

def test():
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
      for data, target in validation_dataloader:
          data, target = data.to(device), target.to(device)
          output = model(data)
          test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
          pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
          correct += pred.eq(target.view_as(pred)).sum().item()
  test_loss /= len(validation_dataloader.dataset)

  print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
      test_loss, correct, len(validation_dataloader.dataset),
      100. * correct / len(validation_dataloader.dataset)))
  return 100. * correct / len(validation_dataloader.dataset)

torch.Size([32, 1, 450, 450])


In [0]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    addition = 20

    self.cl1 = nn.Conv2d(1,32,3,1,padding=2)
    self.cl2 = nn.Conv2d(32,32,3,1,padding=1)
    self.cl3 = nn.Conv2d(32,64,3,1,padding=1)
    self.cl4 = nn.Conv2d(64,64,3,1,padding=1)
    self.cl5 = nn.Conv2d(64,128,3,1,padding=1)
    self.cl6 = nn.Conv2d(128,128,3,1,padding=1)
    self.cl7 = nn.Conv2d(128,256,3,1,padding=1)
    self.cl8 = nn.Conv2d(256,256,3,1,padding=1)
    self.fc1 = nn.Linear(256 * (28**2), 256)
    self.fc2 = nn.Linear(256, 256)
    self.fc3 = nn.Linear(256, 10)
  def forward(self, x): 
    x = F.relu(self.cl1(x))
    # print(x.shape)
    x = F.relu(self.cl2(x))
    # print(x.shape)
    x = F.max_pool2d(x, 2, 2)
    # print(x.shape)
    x = F.relu(self.cl3(x))
    # print(x.shape)
    x = F.relu(self.cl4(x))
    # print(x.shape)
    x = F.max_pool2d(x, 2, 2)
    # print(x.shape)
    x = F.relu(self.cl5(x))
    # print(x.shape)
    x = F.relu(self.cl6(x))
    # print(x.shape)
    x = F.max_pool2d(x, 2, 2)
    # print(x.shape)
    x = F.relu(self.cl7(x))
    # print(x.shape)
    x = F.relu(self.cl8(x))
    # print(x.shape)
    x = F.max_pool2d(x, 2, 2)
    # print(x.shape)
    x = x.view(-1, self.fc1.in_features)
    x = F.relu(self.fc1(x))
    x = F.dropout2d(x)
    x = F.relu(self.fc2(x))
    x = F.dropout2d(x)
    x = self.fc3(x)
    x = torch.sigmoid(x) # used dim=1 to work on the data not on the batch
    return x
model = Net().to(device)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr=0.0001)#.Adam(model.parameters(), lr=0.002, amsgrad=True)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[500,1000,1500], gamma=0.5)

In [19]:
gc.collect()

0

In [0]:
epochs = 20
itr = 1
p_itr = 10
total_loss = 0
loss_list = []
acc_list = []
for epoch in range(epochs):
    gc.collect()
    model.train()
    for samples, labels in dataloader:
        samples, labels = samples.to(device), labels.to(device)
        optimizer.zero_grad()
        output = model(samples)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        scheduler.step()
        
        if itr%p_itr == 0:
            pred = torch.argmax(output, dim=1)
            correct = pred.eq(labels)
            acc = torch.mean(correct.float())
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, acc))
            loss_list.append(total_loss/p_itr)
            acc_list.append(acc)
            total_loss = 0
            
        itr += 1
    acc = test()
    torch.save(model.state_dict(),"/content/drive/My Drive/datasets/dogscats/cnn3.pt")
    if (acc > 85):
      break
    for param_group in optimizer.param_groups:
      param_group['lr'] = param_group['lr'] * 0.95
print(itr)


[Epoch 1/20] Iteration 10 -> Train Loss: 1.7531, Accuracy: 0.750
[Epoch 1/20] Iteration 20 -> Train Loss: 1.6116, Accuracy: 0.531
[Epoch 1/20] Iteration 30 -> Train Loss: 1.6065, Accuracy: 0.438
[Epoch 1/20] Iteration 40 -> Train Loss: 1.6042, Accuracy: 0.562
[Epoch 1/20] Iteration 50 -> Train Loss: 1.6087, Accuracy: 0.562
[Epoch 1/20] Iteration 60 -> Train Loss: 1.6029, Accuracy: 0.344
[Epoch 1/20] Iteration 70 -> Train Loss: 1.6014, Accuracy: 0.500
[Epoch 1/20] Iteration 80 -> Train Loss: 1.6004, Accuracy: 0.469
[Epoch 1/20] Iteration 90 -> Train Loss: 1.6021, Accuracy: 0.469
[Epoch 1/20] Iteration 100 -> Train Loss: 1.5995, Accuracy: 0.500
[Epoch 1/20] Iteration 110 -> Train Loss: 1.5995, Accuracy: 0.531
[Epoch 1/20] Iteration 120 -> Train Loss: 1.5981, Accuracy: 0.562
[Epoch 1/20] Iteration 130 -> Train Loss: 1.6015, Accuracy: 0.375
[Epoch 1/20] Iteration 140 -> Train Loss: 1.5983, Accuracy: 0.531
[Epoch 1/20] Iteration 150 -> Train Loss: 1.5991, Accuracy: 0.594
[Epoch 1/20] Iterat

In [0]:
criterion = nn.CrossEntropyLoss()